<a href="https://colab.research.google.com/github/wangjh789/2021-2_BigData/blob/master/brute_force.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/gohdong/2021_autumn/master/BigData/project1/facebook_combined.txt 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz 
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install findspark
!pip install pyspark

--2021-10-07 15:53:17--  https://raw.githubusercontent.com/gohdong/2021_autumn/master/BigData/project1/facebook_combined.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 854362 (834K) [text/plain]
Saving to: ‘facebook_combined.txt.3’

facebook_combined.t 100%[===================>] 834.34K  --.-KB/s    in 0.02s   

2021-10-07 15:53:17 (45.9 MB/s) - ‘facebook_combined.txt.3’ saved [854362/854362]

--2021-10-07 15:53:19--  https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224445805 (214M) [application/x-gzip]
Saving to: ‘s

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext


sc = SparkContext("local","bf")

In [3]:
undirect = sc.textFile('facebook_combined.txt').map(lambda x : x.split(" "))\
.flatMap(lambda x: ((int(x[0]),int(x[1])),(int(x[1]),int(x[0])))).groupByKey().mapValues(list)

threshold = 0.6

In [4]:
ref = sc.broadcast(dict(undirect.collect()))

In [5]:
def make_invert(item):
    key= item[0]
    arr = item[1]
    result = []
    for i in arr:
        result.append((i,key))
    return result

In [6]:
invert = undirect.flatMap(make_invert).groupByKey().mapValues(list)
invert.count()

4039

In [7]:
def make_pairs(item):
    result = []
    arr = item[1]
    for idx,i in enumerate(arr):
        for j in arr[idx+1:]:
            result.append(((i,j),1))
    return result

In [8]:
pairs = invert.flatMap(make_pairs).reduceByKey(lambda x,y : x+y)
pairs.count()

1446223

In [9]:
def compute_overlap(item):
    x = item[0][0]
    y = item[0][1]
    overlap = item[1]
    a = threshold/(1+threshold)*(len(ref.value[x])+len(ref.value[y]))
    return overlap >= a

def is_not_friend(item):
    x = item[0][0]
    y = item[0][1]
    return not (x in ref.value[y])

In [10]:
result = pairs.filter(compute_overlap).filter(is_not_friend).flatMap(lambda x: ((x[0][0],x[0][1]),(x[0][1],x[0][0])))
result.count()

3238